In [4]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, f1_score, accuracy_score, classification_report
from collections import Counter 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [5]:
data = pd.read_excel('H:\DATA\MY\practice\Practice-25\Pumpkin_Seeds_Dataset.xlsx')
data.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Convex_Area,Equiv_Diameter,Eccentricity,Solidity,Extent,Roundness,Aspect_Ration,Compactness,Class
0,56276,888.242,326.1485,220.2388,56831,267.6805,0.7376,0.9902,0.7453,0.8963,1.4809,0.8207,Çerçevelik
1,76631,1068.146,417.1932,234.2289,77280,312.3614,0.8275,0.9916,0.7151,0.8440,1.7811,0.7487,Çerçevelik
2,71623,1082.987,435.8328,211.0457,72663,301.9822,0.8749,0.9857,0.7400,0.7674,2.0651,0.6929,Çerçevelik
3,66458,992.051,381.5638,222.5322,67118,290.8899,0.8123,0.9902,0.7396,0.8486,1.7146,0.7624,Çerçevelik
4,66107,998.146,383.8883,220.4545,67117,290.1207,0.8187,0.9850,0.6752,0.8338,1.7413,0.7557,Çerçevelik


In [6]:
print(data.shape)
print(data.info())

(2500, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               2500 non-null   int64  
 1   Perimeter          2500 non-null   float64
 2   Major_Axis_Length  2500 non-null   float64
 3   Minor_Axis_Length  2500 non-null   float64
 4   Convex_Area        2500 non-null   int64  
 5   Equiv_Diameter     2500 non-null   float64
 6   Eccentricity       2500 non-null   float64
 7   Solidity           2500 non-null   float64
 8   Extent             2500 non-null   float64
 9   Roundness          2500 non-null   float64
 10  Aspect_Ration      2500 non-null   float64
 11  Compactness        2500 non-null   float64
 12  Class              2500 non-null   object 
dtypes: float64(10), int64(2), object(1)
memory usage: 254.0+ KB
None


In [7]:
data = data.round(2)
print(data.isnull().sum())
print(Counter(data['Class']))

Area                 0
Perimeter            0
Major_Axis_Length    0
Minor_Axis_Length    0
Convex_Area          0
Equiv_Diameter       0
Eccentricity         0
Solidity             0
Extent               0
Roundness            0
Aspect_Ration        0
Compactness          0
Class                0
dtype: int64
Counter({'Çerçevelik': 1300, 'Ürgüp Sivrisi': 1200})


In [8]:
# the data overall seems to be in a good quality , and it is ready to go
# the type of model we are performing is random forest and KNN 
# the target variable does not suffer from the imbalanced data issue

model = RandomForestClassifier(random_state=42)

X = data.drop('Class',axis = 1)
Y = data['Class']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state= 42)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print('the accuracy score of the model is : ', accuracy)
print('the classification report is : ',report)

the accuracy score of the model is :  0.85
the classification report is :                 precision    recall  f1-score   support

   Çerçevelik       0.84      0.87      0.85       251
Ürgüp Sivrisi       0.86      0.83      0.85       249

     accuracy                           0.85       500
    macro avg       0.85      0.85      0.85       500
 weighted avg       0.85      0.85      0.85       500



In [9]:
# the model performance seems to be fine , let's confirm it white the cross validation

cr_value = cross_val_score(model, X, Y, cv= 5 , scoring='accuracy')

print('the cross validation : ', cr_value)
print('the mean cross validation score :',cr_value.mean())

the cross validation :  [0.9   0.892 0.882 0.856 0.872]
the mean cross validation score : 0.8804000000000001


In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
  'n_estimators': [100, 200, 300],
  'max_depth': [None, 10, 20, 30],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4],
  'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator= model, param_grid=param_grid, cv = 5, n_jobs=-1, verbose=2)

grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(x_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\mouncef\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
401 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mouncef\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\mouncef\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\mouncef\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\mouncef\anaconda3\Lib\

Best parameters found: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy: 0.85
Classification Report:
               precision    recall  f1-score   support

   Çerçevelik       0.84      0.86      0.85       251
Ürgüp Sivrisi       0.86      0.84      0.85       249

     accuracy                           0.85       500
    macro avg       0.85      0.85      0.85       500
 weighted avg       0.85      0.85      0.85       500



In [11]:
model = KNeighborsClassifier(n_neighbors = 15)
cr_value = cross_val_score(model, X, Y, cv= 5 , scoring='accuracy')

print('the cross validation : ', cr_value)
print('the mean cross validation score :',cr_value.mean())

the cross validation :  [0.614 0.608 0.606 0.588 0.608]
the mean cross validation score : 0.6048


In [12]:
param_grid = {
    'n_neighbors': [5, 7, 9, 10, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(x_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Use the best model to predict on the test set
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(x_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters found: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Accuracy: 0.742
Classification Report:
               precision    recall  f1-score   support

   Çerçevelik       0.71      0.82      0.76       251
Ürgüp Sivrisi       0.78      0.67      0.72       249

     accuracy                           0.74       500
    macro avg       0.75      0.74      0.74       500
 weighted avg       0.75      0.74      0.74       500



In [13]:
from sklearn.svm import SVC

model = SVC(kernel='rbf')

cr_value = cross_val_score(model, X, Y, cv= 5 , scoring='accuracy')

print('the cross validation : ', cr_value)
print('the mean cross validation score :',cr_value.mean())

the cross validation :  [0.578 0.634 0.566 0.554 0.572]
the mean cross validation score : 0.5808


In [14]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Initialize StratifiedKFold with a fixed random state
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with StratifiedKFold
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=cv, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters from GridSearchCV
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Use the best model to predict on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the best model
print(f"Accuracy with Best Model: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report with Best Model:\n{classification_report(y_test, y_pred)}")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'reg_lambda': 2}
Accuracy with Best Model: 0.85
Classification Report with Best Model:
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       251
           1       0.86      0.83      0.85       249

    accuracy                           0.85       500
   macro avg       0.85      0.85      0.85       500
weighted avg       0.85      0.85      0.85       500



c:\Users\mouncef\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:39:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
